# PyCSEP cybertraining

In this tutorial, we show how use the PyCSEP package to load and evaluate catalog-based forecasts generated using the UCERF3-ETAS model. The procedure is general and can be applied to any other forecasting model producing catalog-based forecasts in the same format. The aim of this tutorial is to illustrate how to produce maps of expected sesmic activity along with observations at different time scales, and to familiarise the user with computing and interpreting CSEP tests results. In doing this, we focus on the difference between grid-based forecasts and catalog-based forecasts, showing that the latter provides more information and more trustworthy results.

Full documnetation of the package can be found [here](https://docs.cseptesting.org/), while any issue can be reported on the [PyCSEP Github page](https://github.com/SCECcode/pycsep).

In [ ]:
#import libraries
import warnings
import os
import json
import time
import itertools
import datetime
import cartopy
# Third-party Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage as nd
import matplotlib.pyplot as pyplot
# import PyCSEP
import csep
from csep.utils import datasets, time_utils, comcat, plots
from csep.core import regions, catalog_evaluations

## Load Forecast

In the following example, we show how to load a catalog-based forecast produced by the UCERF3-ETAS forecasts along with the corresponding observed earthquakes from the ComCat catalog. We focus on the El-Mayor Cucapah 7.2 $M_w$ earthquake occurred in Southern California on the 04, April 2010, and we consider the two forecasts corresponding to the day, and the day after the event.

Before loading a forecast, we need to create a space-magnitude region of interest that will be used to filter all the catalogs composing a catalog-based forecast. This is done by specifying a magnitude grid by providing a minimum and maximum magnitudes, and the length of the magnitude bin, namely $\texttt{min\_mw}$, $\texttt{max\_mw}$, and $\texttt{dmw}$, and a geographical region, $\texttt{region}$, which is provided by the PyCSEP package. Then, we need to define a start and end time of the forecast, namely $\texttt{start\_time}$ and $\texttt{start\_time}$. Then, the forecasts can be loaded using the PyCSEP function $\texttt{load\_catalog\_forecast}$.

The function $\texttt{load\_catalog\_forecast}$ takes (at least) the following arguments:

* $\texttt{fname}$: string indicating the file's path
* $\texttt{start\_time}$: datetime.datetime object indicating the start time of the forecasts
* $\texttt{end\_time}$: datetime.datetime object indicating the end time of the forecasts
* $\texttt{type}$: string indicating the type of forecast file 
* $\texttt{region}$: csep.core.spatial.CartesianGrid2D object representing the space-magnitude region
* $\texttt{filter\_spatial}$: boolean indicating whether catalogs should be filtered in space
* $\texttt{apply\_filters}$: boolean indicating whether catalogs should be filtered using the input in filters
* $\texttt{fname}$: string indicating additional filters

The function returns a csep.core.forecasts.CatalogForecast object. This is a flexible class provided by PyCSEP and useful to efficiently perform computations needed for plotting and evaluating the forecast. More details on the arguments of this function and the csep.core.forecasts.CatalogForecast class can be found [here](https://docs.cseptesting.org/reference/generated/csep.load_catalog_forecast.html).

In [ ]:
#####################
### LOAD FORECAST ###
#####################

min_mw = 3.95 # minimum magnitude
max_mw = 8.95 # max magnitude after which is just one bin
dmw = 0.1 # bin width

# Create space and magnitude regions. The forecast is already filtered in space and magnitude
magnitudes = regions.magnitude_bins(min_mw, max_mw, dmw)
region = regions.california_relm_region()

# Combine space and magnitude regions
space_magnitude_region = regions.create_space_magnitude_region(region, magnitudes)


# set start and end date
start_time_event = time_utils.strptime_to_utc_datetime('2010-04-04 00:00:00.0')
end_time_event = time_utils.strptime_to_utc_datetime('2010-04-04 23:59:59.0')
#load forecast
forecast_day_event = csep.load_catalog_forecast(fname = 'forecasts/forecast_04_04_2010.bin',
                                                 start_time = start_time_event,
                                                 end_time = end_time_event,
                                                 type='ucerf3',
                                                 region = space_magnitude_region,
                                                 filter_spatial = True,
                                                 apply_filters = True,
                                                 filters = 'magnitude >= 3.95')

## Visualise a forecast
To visualise the spatial distribution of a catalog-based forecast it is convenient to retrieve the average number of events per catalog for each cell of a pre-defined space-magnitude grid. The function $\texttt{get\_expected\_rates}$ does just this and returns a $\texttt{csep.core.forecasts.GriddedForecast}$ object. This is a class provided by PyCSEP to represent grid-based forecast, and contains a number of tools to analyse and access information on the average number of events per cell. 

In [ ]:
#calculate expected rates per space-magnitude cell
expected_rates_event = forecast_day_event.get_expected_rates(verbose=True) # 220-250 secs on standard laptop

Once we have obtained the grid-based forecast associated with a catalog-based forecast, we can use PyCSEP functionalities to visualise it. In the code below, firstly, we set the plot options, we generate the figure, and we save it to pdf. We use the following plotting options:

* $\texttt{title}$: string representing the title of the figure.
* $\texttt{borders (coastlines)}$: boolean indicating whether the borders of the region (the coastlines) should be included.
* $\texttt{basemap}$: string representing the type of base map used as background, possible values are : stock_img, stamen_terrain, stamen_terrain-background, google-satellite, ESRI_terrain, ESRI_imagery, ESRI_relief, ESRI_topo, ESRI_terrain.
* $\texttt{projection}$: Projection to be used in the basemap.
* $\texttt{cmap}$: string indicating the color palette, typical options are: rainbow, plasma, inferno, viridis, seismic. For a complete list we refer to the [website](https://matplotlib.org/stable/users/explain/colors/colormaps.html).
* $\texttt{clim}$: list of two elements representing the extremes of the color scale.

The full list of figure options can be found [here](https://docs.cseptesting.org/tutorials/plot_customizations.html)

In [ ]:
# arguments to plot the forecast
args_forecast = {'title': 'El-Mayor Cucapah 04/04/2010 forecast', # title of the plot
                 'borders': True, # adding borders of the region
                 'coastline': True, # adding coastline
                 'basemap': 'google-satellite', # type of background map
                 'projection': cartopy.crs.Mercator(), # projection
                 'cmap': 'seismic', # color palette
                 'clim':[-9, 0]} # limits of the color scale
# plot 
expected_rates_event.plot(plot_args=args_forecast)
# save the plot to pdf 
plt.savefig('data/outputs/ElMayor_forecast_spatdistro.pdf')

## Load and visualise observations
PyCSEP also provides access to the [ComCat](https://earthquake.usgs.gov/data/comcat/) earthquake catalogue. To download the data relative to a time period it is sufficient to specify the start and end dates. We are interested in the catalog corresponding to the 04, April 2010, the day of the $7.2 M_w$ El-Mayor Cucapah earthquake. With the code below, we load and filter the data. The code returns a $\texttt{csep.core.catalogs.CSEPCatalog()}$ object which is the standard class to represent catalogs adopted in PyCSEP providing easy access to different statistics and functionalities. For example, just by printing the catalog we can see that it contains 18 events with corresponding time, space, and magnitude ranges.

In [ ]:
#########################
### LOAD OBSERVATIONS ###
#########################
# retrieve events in ComCat catalogue between start and end date
catalog = csep.query_comcat(start_time_event, end_time_event)
# filter magnitude below 3.95
catalog = catalog.filter('magnitude >= 3.95')
# filter events outside spatial region
catalog = catalog.filter_spatial(space_magnitude_region)
# print summary information 
print(catalog)

To visualise the catalog, similarly to the forecast, we can set different figure options. A full list of arguments can be found [here](https://docs.cseptesting.org/tutorials/plot_customizations.html)

In [ ]:
# argumnets to plot a catalog
args_catalog = {'basemap': 'ESRI_terrain',
                'markercolor': 'red',
                'markersize': 1,
                'alpha':0.5}
catalog.plot(plot_args=args_catalog)

### Plot forecast and observations
We can also superimpose the observations to the forecast plot obtained earlier. To do this, we need to create the forecast figure as before, and passing it as input for the catalog plot along with the others plotting arguments.

In [ ]:
# superimpose points and forecast
ax1 = expected_rates_event.plot(plot_args = args_forecast)
args_catalog['markercolor'] = 'black' # change color of observations
args_catalog['title'] = 'El-Mayor Cucapah 7.2 M, 04/04/2010' # set title
ax2 = catalog.plot(ax=ax1, plot_args = args_catalog)

## Visualise magnitude distribution
The $\texttt{csep.core.forecasts.GriddedForecast}$ format provide access to the average magnitude distribution provided by the forecast. For each magnitude cell, it provides the average number of events per synthetic catalog. We can access the average number of events per cell with $\texttt{[forecast].magnitude\_counts()}$ and the magnitude cells' centroids with $\texttt{[forecast].magnitudes}$. In the same way, the $\texttt{csep.core.catalogs.CSEPCatalog()}$ format provides access to the number of observed events per magnitude cell with the command $\texttt{[catalog].magnitude\_counts()}$. The magnitude grid is the same as we have used the same space-magnitude region to define both the forecast and the catalog.  

In [ ]:
# access average number of synthetic events per catalog per magnitude cell
print(expected_rates_event.magnitude_counts())
# access number of observed events per magnitude cell
print(catalog.magnitude_counts())
# access cells' centroids
print(expected_rates_event.magnitudes)

Having access to the expected and observed number of events per magnitude cell, we can visualise them as it is usually done for magnitude distributions. We can see in this case that the observed distribution is strongly influenced by the presence of the $7.2 M_w$ event, this is normal given we are analysing just one day of seismicity. The forecast presents the linear shape of a Gutenberg-Richter law with $b = 1$. The departure from the line at large magnitudes is due to the limited samples. 

In [ ]:
warnings.filterwarnings('ignore') 
# calculate nomralised rates per magnitude cell - forecast
norm_forecast_mag_rates = expected_rates_event.magnitude_counts()/np.sum(expected_rates_event.magnitude_counts())
# calculate logarithm - forecast
log_norm_forecast_mag_rates = np.log10(norm_forecast_mag_rates)

# calculate nomralised rates per magnitude cell - forecast
norm_observed_mag_rates = catalog.magnitude_counts()/np.sum(catalog.magnitude_counts())
# calculate logarithm - forecast
log_norm_observed_mag_rates = np.log10(norm_observed_mag_rates)

b_value = 1
# plot
plt.scatter(space_magnitude_region.magnitudes, log_norm_forecast_mag_rates, label = 'forecast - 04/04/2010', s = 4)
plt.scatter(space_magnitude_region.magnitudes, log_norm_observed_mag_rates, label = 'observation - 04/04/2010', marker = 'x')
plt.axline((space_magnitude_region.magnitudes[0],
           log_norm_forecast_mag_rates[0]), slope= -b_value, label='GR b = 1', linewidth = 1, alpha = 0.75)
plt.ylabel('log10 M3.95+ rate per cell')
plt.xlabel('M')
plt.legend(framealpha = 0.75)
plt.show()

## Evaluate a forecast with CSEP consistency tests
The PyCSEP package provides access to a suite of statistical tests to evaluate various aspects of the forecast. In this tutorial, we show how to compute and interpret the N-test for the number of events and the S-test for the spatial distribution. For a full list of the available tests for catalog-based foercasts please visit the [website](https://docs.cseptesting.org/concepts/evaluations.html#module-csep.core.catalog_evaluations). 

All the CSEP tests for catalog-based forecasts are based on the same logic. A statistics is calculated for every synthetic catalog composing the forecast, this is the number of events for the N-test, while is a normalised approximated log-likelihood for the S-test (see [Savran et al. 2020](https://docs.cseptesting.org/reference/publications.html#savran-2020)). This produce a number (the value of the statistic) for each synthetic catalog, we refer to this set of values as the test distribution. This is because these values are used to estimate the distribution of the statistic under the model. For example, for the N-test, we use the number of events per catalog to estimate the distribution of the number of events under (provided by) the forecast. To conclude, the same statistic is calculated on the observed catalog and compared with the test distribution. 

We show how to perform the comparison between the test distribution and the observed statistic visually, by plotting them one against the other, and numerically, by calculating the fraction of synthetic catalogs with statistic greater or smaller then the observed one. We refer to this value as $\delta$; values of $\delta$ close to 0 or 1 are indicative of poor fit as the synthetic catalogs provide values of the statistic consistently smaller or greater than the observations.  

### N-test
We can run CSEP tests for catalog based forecasts with the command $\texttt{catalog\_evaluations.[test\_name]}$. The code below is for the N-test which only requires a forecast and a catalog, respectively in the $\texttt{csep.core.forecasts.GriddedForecast}$ and $\texttt{csep.core.catalogs.CSEPCatalog()}$ format, to be computed. We notice that the forecast and the catalog have to be defined on the same space magnitude region for the function to work. The function $\texttt{catalog\_evaluations.number\_test()}$ calculates the number of events for the synthetic and the observed catalogs and returns the results in the $\texttt{csep.models.CatalogNumberTestResult}$ format, which is a format provided by PyCSEP to represent test results. 

In [ ]:
number_test_result = catalog_evaluations.number_test(forecast_day_event, catalog) # 0.49332284927368164

Once we have computed the test result we can visualise them with the $\texttt{[test\_results].plot()}$ function which, as for catalogs and forecasts, can be customised with a list of arguments. In this case, the plot is degenerate as the distribution of the number of events is strongly clustered on the 0. The red bars represents regions outside the $95\%$ of the test distribution, the value $95$ can be regulated by the user using the $\texttt{percentile}$ argument. The vertical dotted line represents the observed statsitic which is 18 as the number of $3.95$+$M_w$ events occurred on 04, April 2010. This is way over the number of events expected by the model which is expected for the first day of the sequence.

In [ ]:
args_n_test = {'title': 'N-test - 04/04/2010', 
           'bins':250,
           'percentile':95}
number_test_result.plot(plot_args = args_n_test)

We can access the test distribution directly with the command $\texttt[test\_result].test_distribution$, this may be useful to calculate summary statistics or to find interesting synthetic catalogs. In the example below, we see that more than $75\%$ of the synthetic catalogs have zero events, and that the maximum number of events is 184. We can find and plot the synthetic catalog with the maximum number of events. The list of synethtic catalogs in $\texttt{csep.core.catalogs.CSEPCatalog()}$ format can be accessed with the command $\texttt{[forecast].catalogs}$. We can see that the synthetic catalog has 184 events with maximum magnitude around $7.7$, the figure shows that the synthetic catalog is strongly clustered around the faults which highlights the spatial features of the UCERF3-ETAS model. 

In [ ]:
# extract test distribution
test_distro = pd.Series(number_test_result.test_distribution)
# print summary statistics
test_distro.describe()

In [ ]:
# find catalog with maximum number of events
catalog_max_n = forecast_day_event.catalogs[test_distro.idxmax()]
# print summary informations
print(catalog_max_n)

# create plot title
plot_title = 'Synthetic catalog with ' + format(np.max(test_distro)) + ' M3.95+ events - 04/04/2010'

# plot arguments
args_catalog_2 = {'title': plot_title,
                'basemap': 'ESRI_terrain',
                'markercolor': 'red',
                'markersize': 1,
                'alpha':0.5}
#visualise synthetic catalog 
catalog_max_n.plot(plot_args = args_catalog_2)

### S-test

To code to run the S-test is similar to the one used for the N-test. In this case, the results indicate that the model produce catalogs with test statistics similar to the observed catalog. The probability that a synthetic catalog has a statistic lower than the observed one is $0.7$, however we can see that this value is strongly influenced by the spike around -10. The strange shape of the distribution is mostly due to the fact that more than $75\%$ of the synthetic catalogs have zero events and provide no information on the spatial distribution. We will explain how to interpret S-test results in the next example.

In [ ]:
# compute S-test
s_test_result = catalog_evaluations.spatial_test(forecast_day_event, catalog, verbose = False)
args_s_test = {'title': 'S-test - 04/04/2010', 
           'bins':25,
           'percentile':95}
s_test_result.plot(plot_args = args_s_test)

# Forecast for the day after El-Mayor Cucapah
Below we repeat the analysis for the day after El-Mayor Cucapah $7.2$ $M_w$ earthquake, namely the 5, April 2010. This day is interesting because has more events (30 3.95+ $M_w$) compared to the 04/04/2010 which provides a useful scenario to interpret the CSEP test results more closely. Also, the forecast now takes into account that a large event just occurred and we can see the different parts of the UCERF3-ETAS model that are activated. We start by plotting the spatial and magnitude distribution of the forecast. The most interesting of the two is the magnitude distribution which is clearly different from the one provided by the forecast for the day before. This is because UCERF3-ETAS incorporates hypothesis on the recurrence of large events, and the probability of large magnitude events is reduced after one. 

In [ ]:
# set start and end date
start_time_after = time_utils.strptime_to_utc_datetime('2010-04-05 00:00:00.0')
end_time_after = time_utils.strptime_to_utc_datetime('2010-04-05 23:59:59.0')
#load forecast
forecast_day_after = csep.load_catalog_forecast(fname = 'forecasts/forecast_05_04_2010.bin', #file path
                                                start_time = start_time_after, # starting time
                                                end_time = end_time_after, # end time
                                                type='ucerf3', # type of forecast 
                                                region = space_magnitude_region, # space magnitude region
                                                filter_spatial = True, # boolean indicating if filtering in space is performed
                                                apply_filters = True, # boolean indicating other filtering is performed
                                                filters = 'magnitude >= 3.95') # additional filters

# retrieve events in ComCat catalogue between start and end date
catalog_after = csep.query_comcat(start_time_after, end_time_after)
# filter magnitude below 3.95
catalog_after = catalog_after.filter('magnitude >= 3.95')
# filter events outside spatial region
catalog_after = catalog_after.filter_spatial(space_magnitude_region)

In [ ]:
expected_rates_after = forecast_day_after.get_expected_rates(verbose=False) 
ax1 = expected_rates_after.plot(plot_args = args_forecast)
args_catalog['title'] = 'El-Mayor Cucapah, 05/04/2010 forecast'
catalog_after.plot(ax = ax1, plot_args = args_catalog)

In [ ]:
# magnitude distributions
# calculate log normalised rates per magnitude cell provided by the forecast
norm_forecast_mag_rates_after = expected_rates_after.magnitude_counts()/np.sum(expected_rates_after.magnitude_counts())
log_norm_forecast_mag_rates_after = np.log10(norm_forecast_mag_rates_after)

# calculate log normalised rates per magnitude cell provided by the catalog
norm_observed_mag_rates_after = catalog_after.magnitude_counts()/np.sum(catalog_after.magnitude_counts())
log_norm_observed_mag_rates_after = np.log10(norm_observed_mag_rates_after)


# plot
plt.scatter(space_magnitude_region.magnitudes, log_norm_forecast_mag_rates, label = 'forecast - 04/04/24', s = 4)
plt.axline((space_magnitude_region.magnitudes[0], log_norm_forecast_mag_rates_after[0]), 
           slope = -b_value, label = 'GR b = 1', linewidth = 1, alpha = 0.75)
plt.scatter(space_magnitude_region.magnitudes, log_norm_forecast_mag_rates_after, label = 'forecast - 05/04/24', s = 4)
plt.scatter(space_magnitude_region.magnitudes, log_norm_observed_mag_rates_after, label = 'observation - 05/04/24', marker = 'x')
plt.ylabel('log10 M3.95+ rate per cell')
plt.xlabel('M')
plt.legend(framealpha = 0.75)
plt.legend()
plt.show()


## CSEP consistency tests
We run the N- and S-test also for this forecast. We see that the test distributions in both cases are more regular than before. This is due to the fact that we have less synthetic catalogs with no observations. 

### N-test
Looking at the N-test distribution we see that less than the $25\%$ of the synthetic catalogs are empty while before it was over $75\%$. Looking at the synthetic catalog with the most number of events, we see that differently from before the events are not clustered along the faults but rather in a cloud shape. The difference is due to the different parts of the UCERF3-ETAS which are activated. In the case of the 4 April, 2010 the points are strongly clustered around the faults because the background part of the model is active given that there were no large events before. Contrarily, in the case of the 5 April, 2010 the aftershock part of the model is active which does not account for faults. 

In [ ]:
number_test_result_after = csep.catalog_evaluations.number_test(forecast_day_after, catalog_after, verbose = False)
number_test_result_after.plot()

In [ ]:
n_test_distribution_after = pd.Series(number_test_result_after.test_distribution)
print(n_test_distribution_after.describe())
catalog_max_n_after = forecast_day_after.catalogs[n_test_distribution_after.idxmax()]
args_catalog_2['title'] = 'Synthetic catalog with '+ format(np.max(n_test_distribution_after)) + ' M3.95+ events - 05/04/2010'
print(catalog_max_n_after)
catalog_max_n_after.plot(plot_args = args_catalog_2)

### S-test
Also for the S-test, the test distribution is more regular than the previous case. In this case, we see that synthetic catalogs often have higher values of the S-test statistics than the observed one ($94\%$ of the cases). This indicates that the synthetic catalogs are too clustered with respect the observations. This is because they present higher log-likelihoods, which means that they tend to be more clustered in the bright area of the map than the observations. If it was the opposite situation, the bulk of the test distribution is on the left side of the observed statistic, we would have the opposite situation, that the forecast is too smooth. Similarly, this is due to the fact that the observations tend to cluster more closely in a high likelihood region than the synthetic catalogs. 

In [ ]:
spatial_test_result_after = csep.catalog_evaluations.spatial_test(forecast_day_after, catalog_after, verbose = False)
spatial_test_result_after.plot()